<a href="https://colab.research.google.com/github/2003Yash/sentence-embedding/blob/main/Sentence_Embedding_Code_for_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as n

In [ ]:
english_file = 'drive/MyDrive/translation_en_kn/train.en'
kannada_file = 'drive/MyDrive/translation_en_kn/train.kn'

START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

#like telugu it's a aplha-syllable languge i.e.., each letter represents a syllable
kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                      'ँ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ',
                      'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ', 'ಌ', 'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ',
                      'ಕ', 'ಖ', 'ಗ', 'ಘ', 'ಙ',
                      'ಚ', 'ಛ', 'ಜ', 'ಝ', 'ಞ',
                      'ಟ', 'ಠ', 'ಡ', 'ಢ', 'ಣ',
                      'ತ', 'ಥ', 'ದ', 'ಧ', 'ನ',
                      'ಪ', 'ಫ', 'ಬ', 'ಭ', 'ಮ',
                      'ಯ', 'ರ', 'ಱ', 'ಲ', 'ಳ', 'ವ', 'ಶ', 'ಷ', 'ಸ', 'ಹ',
                      '಼', 'ಽ', 'ಾ', 'ಿ', 'ೀ', 'ು', 'ೂ', 'ೃ', 'ೄ', 'ೆ', 'ೇ', 'ೈ', 'ೊ', 'ೋ', 'ೌ', '್', 'ೕ', 'ೖ', 'ೞ', 'ೣ', 'ಂ', 'ಃ',
                      '೦', '೧', '೨', '೩', '೪', '೫', '೬', '೭', '೮', '೯', PADDING_TOKEN, END_TOKEN]

#english is a phonetic alphabet i.e.., we commbine letter to make a syllable
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                        'Y', 'Z',
                        '[', '\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
text = 'ಕನ್ನಡ'
list(text)

In [ ]:
'ಕ' + 'ಾ'

In [ ]:
# we are teaching the model to train for each letter not a word since words might create unneccesary complexity in inference and letter inference is a lot faster
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)} #mapping kanada letter to an index
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)} #vice versa
index_to_english = {k:v for k,v in enumerate(english_vocabulary)} #mapping english letter to an index
english_to_index = {v:k for k,v in enumerate(english_vocabulary)} #vice versa
#we will create english vector and we will not find similar kanada vector instead as we learnt we will find the next most probable vector for that vector.

In [ ]:
#reading datasets from g-drive
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(kannada_file, 'r') as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences to consider
TOTAL_SENTENCES = 100000
english_sentences = english_sentences[:TOTAL_SENTENCES] #picking first 100,000 sentences from eng dataset
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES] #picking first 100,000 sentences from kanada dataset
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [ ]:
english_sentences[:10] #printing first 10 sentences

In [ ]:
kannada_sentences[:10] #printing first 10 sentences

In [ ]:
max(len(x) for x in kannada_sentences), max(len(x) for x in english_sentences), #finding max number of sentences in both the sentences

In [ ]:
PERCENTILE = 97
#dist based on sentence length we get 97 percentile length i.e.., 172 i.ee., each dataset have 97% data with less than 172 character or sentence length
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in kannada_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

In [ ]:
#based on above percentile we set the value to 200
max_sequence_length = 200

def is_valid_tokens(sentence, vocab): #valid token if the chars of sentence match with varnamala / aplhabets with no foreign chars
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length): # if sentence length is more than 200 chars then it's invalid
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space


valid_sentence_indicies = []# we will count sentences only if they have valid_tokens in valid_length
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")  #original dataset length before checking validity
print(f"Number of valid sentences: {len(valid_sentence_indicies)}") #kanada dataset length after checking validity

In [ ]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]  #creating a array of valid kanada sentences from valid_sentence array
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]  #creating a array of valid eng sentences from valid_sentence array

In [ ]:
kannada_sentences[:3] # just printing the first three sentences from valid kanada dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

#using pytorch dataset fucntion we can create dataset without too much code
class TextDataset(Dataset):
    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

# we will find and batch the kanada and english sentences together

In [ ]:
dataset = TextDataset(english_sentences, kannada_sentences) #joining both sentences

In [ ]:
len(dataset)

In [ ]:
dataset[1] #printing 2nd dataset value which is a english and it's corresponding kanada sentence

In [ ]:
batch_size = 3 # each time we tune our hyperparams for 3 sentences i.e.., we take account for loss of all 3 rows and we average them and backpropogate that avg loss based on differentiation
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
  #making batch by breaking batch maker when there are 3 items in the batch
    print(batch)
    if batch_num > 3:
        break

In [ ]:
def tokenize(sentence, language_to_index, start_token=True, end_token=True):  #we put satrt and end tokens for the embedding
  # takes the character to number embedding from 4th cell and use it tokenize  the while sentence
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START_TOKEN]) #if start token is found push it to starting char place
    if end_token:
        sentence_word_indicies.append(language_to_index[END_TOKEN]) # if end token is found push it to last char place
    for _ in range(len(sentence_word_indicies), max_sequence_length): # add padding token in the rest of sentence gap
        sentence_word_indicies.append(language_to_index[PADDING_TOKEN])
    return torch.tensor(sentence_word_indicies)

In [ ]:
batch # prints the rows and their translation for the current batch

In [ ]:
batch[sentence_num] #prints the 3 english sentences without their translation

In [ ]:
eng_tokenized, kn_tokenized = [], [] # this line gives all tokeninzed eng and kanada chars
for sentence_num in range(batch_size): #tokenize them as per their language token
    eng_sentence, kn_sentence = batch[0][sentence_num], batch[1][sentence_num]
    eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=False, end_token=False) )
    kn_tokenized.append( tokenize(kn_sentence, kannada_to_index, start_token=True, end_token=True) )
eng_tokenized = torch.stack(eng_tokenized)
kn_tokenized = torch.stack(kn_tokenized)


In [ ]:
eng_tokenized #prints the 3 eng tokenized sentences

In [ ]:
NEG_INFTY = -1e9 # to mask the attention vector,
#we use this low value instead of -int is because this value in future is exponentially calculated for softmax
# so we take low since -int in exp is 0 and if all rows of soft max are zero this results in NAN (not a number) error in model and model crashes

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    print(f"encoder_self_attention_mask {encoder_self_attention_mask.size()}: {encoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_self_attention_mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :10, :10]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


In [ ]:
3create_masks(batch[0], batch[1]) #every batch has 2 masks
                                  #encoder mask which is basically all zeros all open to look ahead
                                  #decoder mask which is all zeros in lower and on principal axis and -1e9 (technically -inf) on above principal axis

In [ ]:
#just one combined code for sentence embedding
class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token=True, end_token=True):

        def tokenize(sentence, start_token=True, end_token=True):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())

         def forward(self, x, end_token=True): # sentence
        x = self.batch_tokenize(x ,end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x
